## 使用函数

In [1]:
from operator import itemgetter  # 导入itemgetter函数，用于从字典中提取值

from langchain_core.prompts import ChatPromptTemplate  # 导入聊天提示模板
from langchain_core.runnables import RunnableLambda  # 导入可运行的Lambda函数包装器
from langchain_deepseek import ChatDeepSeek  # 导入DeepSeek聊天模型
import os  # 导入os模块，用于从环境变量获取API密钥

# 初始化DeepSeek大语言模型
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-R1",  # 使用DeepSeek-R1模型
    temperature=0,  # 设置温度为0，使输出更确定性
    api_key=os.environ.get("DEEPSEEK_API_KEY"),  # 从环境变量获取API密钥
    api_base=os.environ.get("DEEPSEEK_API_BASE"),  # 从环境变量获取API基础URL
)


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])



# 创建一个简单的聊天提示模板，询问a和b的和
prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

# 构建一个复杂的处理链
chain = (
    {
        # 处理"a"参数:
        # 1. 从输入字典中提取"foo"键的值
        # 2. 将提取的值传递给length_function函数(假设这个函数计算字符串长度)
        "a": itemgetter("foo") | RunnableLambda(length_function),
        
        # 处理"b"参数:
        # 1. 创建一个包含两个键值对的字典:
        #    - "text1": 从输入字典中提取"foo"键的值
        #    - "text2": 从输入字典中提取"bar"键的值
        # 2. 将这个字典传递给multiple_length_function函数
        #    (假设这个函数计算两个文本的总长度)
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt  # 将处理后的"a"和"b"值填入提示模板
    | model  # 将填充后的提示发送给DeepSeek模型生成回答
)

# 调用链处理流程，输入一个包含"foo"和"bar"键的字典
# 整个过程:
# 1. 计算"bar"字符串的长度作为a的值
# 2. 计算"bar"和"gah"字符串的总长度作为b的值
# 3. 将这些值填入提示"what is {a} + {b}"
# 4. 让DeepSeek模型回答这个问题
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='The sum of 3 and 9 is calculated as follows:\n\n3 + 9 = 12\n\n**Answer:** 12', additional_kwargs={'refusal': None, 'reasoning_content': "Okay, so I need to figure out what 3 plus 9 is. Let me start by recalling how addition works. Addition is combining two numbers to get their total sum. So if I have 3 of something and then add 9 more, how many do I have in total?\n\nLet me count on my fingers. Starting with 3, if I add 9 more, I can count up 9 numbers from 3. So 3... then 4, 5, 6, 7, 8, 9, 10, 11, 12. That's 9 numbers added, so 3 + 9 should be 12. Wait, let me check that again to make sure I didn't skip a number. Starting at 3, adding 1 gets me to 4 (that's 1), then 5 (2), 6 (3), 7 (4), 8 (5), 9 (6), 10 (7), 11 (8), 12 (9). Yep, that's 9 steps, so 3 + 9 equals 12.\n\nAlternatively, I know that 9 is one less than 10. So maybe breaking it down: 3 + 9 is the same as 3 + (10 - 1). That would be 3 + 10 = 13, then subtract 1 to get 12. That also gives me 12. Hmm, same ans